In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import sys
import cv2 as cv
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
combine_fold = 'C:/Users/oryan/Documents/mergers-in-cosmos/cats-to-combine'
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
fig_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/paper-source/figures'
cos_dat_fold = 'E:/cosmos-data'

In [3]:
df_cat = pd.read_csv(f'{combine_fold}/cosmos2020-categorised-interactions.csv', index_col = 0)

In [4]:
df_cat

,ORyan23_ID,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage3,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,861738,stage1,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,...,-9.574272,-9.548649,-9.503962,-9.439343,-9.384205,0.240042,0.281559,0.330791,0.384371,0.436491
2,4000705539529,1284864,stage1,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,...,-9.669377,-9.614943,-9.570104,-9.533121,-9.476167,0.094187,0.114436,0.140990,0.159430,0.183608
3,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
4,4000705540034,1304538,stage1,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,6000535066150,1157587,stage4,150.422510,2.514655,15373.875977,29370.304688,0.000024,0.000027,8.697323e-06,...,-13.220035,-13.215519,-10.469687,-9.954029,-9.691111,0.173505,0.240862,0.328586,0.439079,0.546386
3792,6000535121452,1035987,stage1,149.694761,2.397231,32824.328125,26552.759766,0.000012,0.000015,-7.174042e-07,...,-10.287872,-10.060802,-9.853634,-9.456022,-9.315375,0.343488,0.463897,0.646257,1.016819,1.285051
3793,6000535865556,1138628,stage1,150.490889,2.496660,13734.171875,28938.828125,0.001929,0.001333,1.953999e-04,...,-9.875473,-9.817194,-9.580817,-9.265499,-9.099182,0.026873,0.026873,0.164250,0.549086,0.751945
3794,6000536013605,1203157,stage1,150.202813,2.558504,20641.730469,30421.910156,0.000051,0.000067,4.840965e-07,...,-9.804204,-9.683063,-9.470870,-9.189605,-8.982660,0.539500,0.595082,0.778112,1.062422,1.232495


In [5]:
df_red_tmp = df_cat.query('ez_sfr < 10')
df_red = df_red_tmp.query('lp_mass_best > 6.5')

### Cross Matching with AGN Catalogues

In [6]:
df_red

,ORyan23_ID,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage3,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,861738,stage1,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,...,-9.574272,-9.548649,-9.503962,-9.439343,-9.384205,0.240042,0.281559,0.330791,0.384371,0.436491
2,4000705539529,1284864,stage1,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,...,-9.669377,-9.614943,-9.570104,-9.533121,-9.476167,0.094187,0.114436,0.140990,0.159430,0.183608
3,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
4,4000705540034,1304538,stage1,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,6000535066150,1157587,stage4,150.422510,2.514655,15373.875977,29370.304688,0.000024,0.000027,8.697323e-06,...,-13.220035,-13.215519,-10.469687,-9.954029,-9.691111,0.173505,0.240862,0.328586,0.439079,0.546386
3792,6000535121452,1035987,stage1,149.694761,2.397231,32824.328125,26552.759766,0.000012,0.000015,-7.174042e-07,...,-10.287872,-10.060802,-9.853634,-9.456022,-9.315375,0.343488,0.463897,0.646257,1.016819,1.285051
3793,6000535865556,1138628,stage1,150.490889,2.496660,13734.171875,28938.828125,0.001929,0.001333,1.953999e-04,...,-9.875473,-9.817194,-9.580817,-9.265499,-9.099182,0.026873,0.026873,0.164250,0.549086,0.751945
3794,6000536013605,1203157,stage1,150.202813,2.558504,20641.730469,30421.910156,0.000051,0.000067,4.840965e-07,...,-9.804204,-9.683063,-9.470870,-9.189605,-8.982660,0.539500,0.595082,0.778112,1.062422,1.232495


### The MPA-JHU

In [8]:
with fits.open(f'{cos_dat_fold}/MPA-JHU_full_catalogue.fits') as hdul:
    mpa_jhu = hdul[1].data

In [20]:
df_coords = (
    df_red[['ID', 'ALPHA_J2000', 'DELTA_J2000']]
    .rename(columns = {'ALPHA_J2000':'RA','DELTA_J2000':'DEC'})
    .set_index('ID')
)

In [21]:
coords_dict = df_coords.to_dict(orient='index')

In [32]:
matched_dict = {}

In [87]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA'] * u.deg, dec = tab['DEC'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return tab[min_ind]['SPECOBJID_1'] 

In [88]:
def matching_ids(ra, dec, mpa_jhu):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = mpa_jhu[(mpa_jhu['RA'] > ra - arcsec_sep_deg) & (mpa_jhu['RA'] < ra + arcsec_sep_deg) & (mpa_jhu['DEC'] > dec - arcsec_sep_deg) & (mpa_jhu['DEC'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return np.nan
    elif len(matching_tab) == 1:
        return matching_tab['SPECOBJID_1']
    elif len(matching_tab) > 1:
        return get_correct_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [89]:
done_ids = list(matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    matched_dict[i] = matching_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], mpa_jhu)

100%|██████████| 3772/3772 [17:13<00:00,  3.65it/s]


In [93]:
df_mpa_matched_lists = pd.DataFrame.from_dict(matched_dict, orient = 'index').rename(columns = {0: 'SpecID_List'})

In [103]:
df_mpa_matched = (
    df_mpa_matched_lists
    .assign(SpecID = df_mpa_matched_lists.SpecID_List.apply(lambda x: x[0] if type(x) == np.chararray else x))
    .dropna()
    .reset_index()
    .drop(columns = 'SpecID_List')
    .rename(columns = {'index' : 'ID'})
)

In [109]:
df_spec_ids = df_mpa_matched.drop_duplicates(subset='SpecID', keep = 'first').set_index('ID')

In [112]:
df_spec_dict = df_spec_ids.to_dict(orient = 'index')

In [142]:
spec_details = {}

In [143]:
def getting_info(specid, mpa_jhu):
    row = mpa_jhu[mpa_jhu['SPECOBJID_1'] == str(specid)]
    columns = row.names
    detail_dict = {}
    for j in columns:
        detail_dict[j] = row[j][0]
    
    return detail_dict

In [144]:
done_ids = list(spec_details.keys())
for i in tqdm(list(df_spec_dict.keys())):
    if i in done_ids:
        continue
    
    spec_details[i] = getting_info(df_spec_dict[i]['SpecID'], mpa_jhu)

100%|██████████| 93/93 [00:30<00:00,  3.10it/s]


In [146]:
spec_details_df = pd.DataFrame.from_dict(spec_details, orient = 'index')

In [150]:
spec_details_df.BPTCLASS.value_counts()

-1    50
 2    15
 1    14
 3     7
 4     4
 5     3
Name: BPTCLASS, dtype: int64

Note for above: The categories are -1 = unclassifiable, 0 = Not used, 1 = Star Forming, 2 - Low S/N Star Forming, 3 = Composite, 4 = AGN Non-Liner, 5 = Low S/N Liner

In [157]:
spec_classification = spec_details_df.query('BPTCLASS >= 1').reset_index().rename(columns = {'index' : 'ID'})[['ID', 'BPTCLASS']]

In [158]:
def classify_ID(clsf):
    if clsf == 1:
        return 'SF'
    elif clsf == 2:
        return 'SF'
    elif clsf == 3:
        return 'Composite'
    elif clsf == 4:
        return 'AGN_Non_Liner'
    elif clsf == 5:
        return 'AGN_Liner'
    else:
        print('What')
        sys.exit()

In [159]:
clsf_df = (
    spec_classification
    .assign(agn_clsf = spec_classification.BPTCLASS.apply(lambda x: classify_ID(x)))
    .drop(columns = 'BPTCLASS')
)

### The VLA 3GHz Counterpart Array

In [166]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_delvecchio_et_al.fits') as hdul:
    vla_cat = hdul[1].data

In [ ]:
vla_cat.names

['ID_VLA3',
 'RA_VLA3',
 'DEC_VLA3',
 'Z_BEST',
 'Z_TYPE',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'flag_herschel3',
 'Mstar',
 'SFR_IR',
 'NUV_R',
 'XRAY_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Radio_excess',
 'Class']

In [197]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_vla_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA3'] * u.deg, dec = tab['DEC_VLA3'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return tab[min_ind]['ID_VLA3']

In [198]:
def matching_vla_ids(ra, dec, vla_cat):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = vla_cat[(vla_cat['RA_VLA3'] > ra - arcsec_sep_deg) & (vla_cat['RA_VLA3'] < ra + arcsec_sep_deg) & (vla_cat['DEC_VLA3'] > dec - arcsec_sep_deg) & (vla_cat['DEC_VLA3'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return np.nan
    elif len(matching_tab) == 1:
        return matching_tab['ID_VLA3'][0]
    elif len(matching_tab) > 1:
        return get_correct_vla_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [199]:
vla_matched_dict = {}

In [200]:
done_ids = list(vla_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    vla_matched_dict[i] = matching_vla_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], vla_cat)

100%|██████████| 3772/3772 [00:19<00:00, 196.63it/s]


In [201]:
vla_matched_tmp = (
    pd.DataFrame.from_dict(vla_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID', 0 : 'VLA_ID_arr'})
)

In [206]:
vla_matched_df = (
    vla_matched_tmp
    .assign(VLA_ID = vla_matched_tmp.VLA_ID_arr.apply(lambda x: int(x[0]) if type(x) == np.array else int(x)))
    .drop(columns = 'VLA_ID_arr')
)

In [207]:
vla_matched_df

,ID,VLA_ID
0,1304538,10560
1,1310001,4870
2,1308927,4870
3,1311422,4870
4,1324485,3284
...,...,...
2824,715824,7887
2825,1157587,10945
2826,1035987,7514
2827,1203157,8349


In [209]:
mpa_ids = list(clsf_df.ID)

In [210]:
vla_checked_df = (
    vla_matched_df
    .assign(in_mpa = vla_matched_df.ID.apply(lambda x: 'y' if x in mpa_ids else 'n'))
)

In [213]:
len(vla_checked_df)

2829

In [214]:
vla_mpa_checked = vla_checked_df.query('in_mpa == "n"')

In [217]:
vla_dedup_df = vla_mpa_checked.drop_duplicates(subset='VLA_ID', keep='first').drop(columns = ['in_mpa'])

In [218]:
vla_dedup_df

,ID,VLA_ID
0,1304538,10560
1,1310001,4870
4,1324485,3284
5,1317610,8832
6,1250257,3714
...,...,...
2793,309053,5074
2797,358919,1091
2800,344439,9291
2814,425321,2022


In [220]:
vla_dict = vla_dedup_df.set_index('ID').to_dict(orient = 'index')

In [261]:
vla_cat_dict = {}

In [268]:
def getting_vla_info(vla_id, vla_cat):
    row = vla_cat[vla_cat['ID_VLA3'] == vla_id]
    ans = row['CLASS'][0]
    
    if len(ans) < 0.5:
        return np.nan
    else:
        return ans

In [269]:
done_ids = list(vla_cat_dict.keys())
for i in tqdm(list(vla_dict.keys())):
    if i in done_ids:
        continue
    
    vla_cat_dict[i] = getting_vla_info(vla_dict[i]['VLA_ID'], vla_cat)

100%|██████████| 1893/1893 [00:06<00:00, 273.22it/s]


In [270]:
vla_cat_df = pd.DataFrame.from_dict(vla_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
vla_cat_df

,clsf
1324485,HLAGN
844303,HLAGN
275831,HLAGN
230625,HLAGN
475667,HLAGN
...,...
630698,MLAGN
623714,HLAGN
1181824,HLAGN
344439,HLAGN


In [271]:
vla_cat_df.clsf.value_counts()

HLAGN    389
MLAGN    270
Name: clsf, dtype: int64

### The Chandra COSMOS Legacy Array

In [275]:
with fits.open(f'{cos_dat_fold}/chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul:
    chandra_rec = hdul[1].data

In [308]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_chandra_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_x'] * u.deg, dec = tab['DEC_x'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return tab[min_ind]['id_x']

In [309]:
def matching_chandra_ids(ra, dec, chandra_rec):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = chandra_rec[(chandra_rec['RA_x'] > ra - arcsec_sep_deg) & (chandra_rec['RA_x'] < ra + arcsec_sep_deg) & (chandra_rec['DEC_x'] > dec - arcsec_sep_deg) & (chandra_rec['DEC_x'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return np.nan
    elif len(matching_tab) == 1:
        return matching_tab['id_x'][0]
    elif len(matching_tab) > 1:
        return get_correct_chandra_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [310]:
chandra_matched_dict = {}

In [311]:
done_ids = list(chandra_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_matched_dict[i] = matching_chandra_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], chandra_rec)

100%|██████████| 3772/3772 [00:47<00:00, 78.93it/s]


In [315]:
chandra_matched_df = pd.DataFrame.from_dict(chandra_matched_dict, orient = 'index').dropna().reset_index().rename(columns = {'index' : 'ID', 0 : 'chandra_id'})

In [319]:
vla_ids = list(vla_cat_df.reset_index()['index'])

In [320]:
chandra_known = (
    chandra_matched_df
    .assign(in_prior = chandra_matched_df.ID.apply(lambda x: 'y' if x in mpa_ids or x in vla_ids else 'n'))
)

In [324]:
chandra_rem = chandra_known.query('in_prior == "n"').drop(columns = 'in_prior')

In [325]:
chandra_dedup = chandra_rem.drop_duplicates(subset='chandra_id', keep='first')

In [326]:
chandra_dedup

,ID,chandra_id
0,1301188,lid_1807
1,1304538,lid_1812
6,1256608,cid_1594
7,1264241,lid_2035
11,280038,lid_2492
...,...,...
1814,1253757,cid_575
1815,1050756,cid_1189
1819,494848,cid_214
1820,883227,cid_2004


In [335]:
def getting_chandra_info(chandra_id, chandra_cat):
    row = chandra_cat[chandra_cat['id_x'] == chandra_id]
    ans = row['spec_type'][0]
    
    
    return ans

In [336]:
chandra_clsf_dict = {}

In [337]:
chandra_dedup_dict = chandra_dedup.set_index('ID').to_dict(orient = 'index')

In [338]:
done_ids = list(chandra_clsf_dict.keys())
for i in tqdm(list(chandra_dedup_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_clsf_dict[i] = getting_chandra_info(chandra_dedup_dict[i]['chandra_id'], chandra_rec)

100%|██████████| 902/902 [00:10<00:00, 85.15it/s]


In [343]:
chandra_clsf_df = pd.DataFrame.from_dict(chandra_clsf_dict, orient = 'index').reset_index().rename(columns = {'index' : 'ID', 0 : 'clsf'})

In [345]:
chandra_clsf_df.clsf.value_counts()

-9     355
 2     279
 1     158
-99     64
-1      28
 0      16
-2       2
Name: clsf, dtype: int64

In [346]:
def classifying_chandra(clsf):
    if clsf == 1:
        return 'BLAGN'
    elif clsf == 2:
        return 'non-BLAGN'
    else:
        return np.nan

In [350]:
chandra_agn_cat = (
    chandra_clsf_df
    .assign(agn_clsf = chandra_clsf_df.clsf.apply(lambda x: classifying_chandra(x)))
    .drop(columns = ['clsf'])
    .dropna()
)

In [351]:
chandra_agn_cat

,ID,agn_clsf
0,1301188,non-BLAGN
1,1304538,non-BLAGN
2,1256608,BLAGN
4,280038,non-BLAGN
6,1075467,BLAGN
...,...,...
893,606519,BLAGN
894,910883,non-BLAGN
897,1253757,non-BLAGN
898,1050756,non-BLAGN


### Putting it all Together!

In [355]:
chandra_agn_cat.head()

,ID,agn_clsf
0,1301188,non-BLAGN
1,1304538,non-BLAGN
2,1256608,BLAGN
4,280038,non-BLAGN
6,1075467,BLAGN


In [ ]:
chandra_agn_cat
vla_cat_df
clsf_df

In [365]:
vla_cat_comb = vla_cat_df.reset_index().rename(columns = {'index' : 'COSMOS_ID', 'clsf' : 'agn_clsf' })

In [366]:
df_agn_cat = pd.concat([clsf_df, vla_cat_comb, chandra_agn_cat])

In [367]:
df_agn_cat.agn_clsf.value_counts()

HLAGN            389
non-BLAGN        279
MLAGN            270
BLAGN            158
SF                29
Composite          7
AGN_Non_Liner      4
AGN_Liner          3
Name: agn_clsf, dtype: int64

In [368]:
df_agn_cat.to_csv(f'{combine_fold}/source-agn-cat.csv')